In [1]:
import sys
import os
import re
import requests
import itertools
from copy import deepcopy
from collections import namedtuple, defaultdict

from bs4 import BeautifulSoup as BS
from tenacity import retry, stop_after_attempt
import simplejson as json

In [2]:
session = requests.Session()
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6",
    "Connection": "keep-alive",
    "Referer": "https://www.google.com.tw/",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) "\
                  "Chrome/69.0.3497.92 Safari/537.36"
}

In [3]:
def issue_page_to_url(issue, page_num):
    url_format = 'https://www.cambridge.org/core/journals/behavioral-and-brain-sciences/' \
                 'issue/{}?pageNum={}'
    return url_format.format(issue, page_num)

def get_website_url(suffix):
    prefix = 'https://www.cambridge.org{}'
    return prefix.format(suffix)

In [4]:
@retry(stop=stop_after_attempt(3))
def get_bsObj(url):
    req = session.get(url, headers=headers)
    bsObj = BS(req.text, "html.parser")
    return bsObj

In [5]:
def get_references_from_url(url):
    
    req = session.get(url, headers=headers)
    art_bsObj = BS(req.text, "html.parser")
    
    reference_infos = []
    for ref in art_bsObj.find_all('span', class_='mixed-citation journal'):
        authors = []
        for name in ref.find_all('span', class_='name'):
            surname = name.find('span', class_='surname')
            surname = surname.text if surname else None
            
            given_names = name.find('span', class_='given-names')
            given_names = given_names.text if given_names else None
            
            detailed_name = DetailedName(surname=surname, given_names=given_names)
            authors.append(detailed_name)

        year = ref.find('span', class_='year')
        year = year.text if year else None
        
        title = ref.find('span', class_='article-title')
        title = title.text if title else None
        
        source = ref.find('span', class_='source')
        source = source.text if source else None

        reference_info = ReferenceInfo(authors=authors,
                                       year=year,
                                       title=title,
                                       source=source)
        reference_infos.append(reference_info)
    return reference_infos

In [6]:
ArticleInfo = namedtuple('ArticleInfo', ['authors', 'title', 'url', 'article_type', 'references'])
AuthorInfo = namedtuple('Author', ['author_name', 'url'])
DetailedName = namedtuple('DetailedName', ['surname', 'given_names'])
ReferenceInfo = namedtuple('ReferenceInfo', ['authors', 'year', 'title', 'source'])

In [7]:
def get_article_infos_from_issue(issue):
    article_infos = []
    for page_num in itertools.count(start=1):

        page = get_bsObj(issue_page_to_url(issue, page_num))

        heading_banner = page.find('h4', class_='journal-article-listing-type heading_12 margin-bottom')
        if not heading_banner:
            break
        assert heading_banner['class'] == ['journal-article-listing-type', 'heading_12', 'margin-bottom']

        now_type = heading_banner.text
        print('\n', now_type, end='\n\n')

        for idx, item in enumerate(heading_banner.find_next_siblings()):
            link = item.find('a', class_='part-link')
            authors = item.find('li', class_='author')
            if link and authors:
                author_infos = []
                for author_objs in authors.find_all('a'):
                    author_name = author_objs.text
                    author_url = author_objs['href']
                    author_infos.append(AuthorInfo(author_name=author_name, url=author_url))

                title = link.text.strip()
                url = get_website_url(link['href'])
                references = get_references_from_url(url)
                info = ArticleInfo(authors=author_infos,
                                   title=title,
                                   url=url,
                                   article_type=now_type,
                                   references=references)
                article_infos.append(info)

                print(title)

            else:
                if item['class'] != ['journal-article-listing-type', 'heading_12', 'margin-bottom']:
                    continue
                now_type = item.text
                print('\n', now_type, end='\n\n')
    return article_infos

In [17]:
def transform_list_to_trees(articles, head='Target Article', children='Open Peer Commentary'):
    head_to_children = defaultdict(dict)
    
    head_article = None
    for article in articles:
        
#         try:
#             article_type = article['article_type']
#         except:
#             article_type = article.article_type
        
        if article['article_type'] == head:

            _article = deepcopy(article)
            title = _article.pop('title', None)
            head_to_children[title] = _article
            head_to_children[title]['commentaries'] = []
            head_article = title

        elif article['article_type'] == children:

            try:
                head_to_children[head_article]['commentaries'].append(article)
            except:
                print(head_to_children[head_article])
                assert False
    
    return head_to_children

In [9]:
# year_to_issues = {2019: ['1F6CCA8ABB7741DAFF471736F58BA234'],
#                   2018: ['03CCA4BACDCFC590CB9C8B2983DC5AB3'],
#                   2017: ['7FEFD73781C19F5897B0CE53B13BD467'],
#                   2016: ['0009F28DF9EDDEA8BE73402A78E14895'],
#                   2015: ['F5C34D98D365BFA85AE25C045B2E7322'],
#                   2014: ['CC2DD914A08BE1D63EB687FB57A6A9D6',
#                          'C6D6667ECBD5CBD182395AD27A612B4A',
#                          'FDCDD1ACF1E2E40A065852ADFE8495B2',
#                          'E1E992159A60720C0B28C1AA80A1EFB0',
#                          '1E0FF2AD50EC066C4E30E46CCC8359E3',
#                          'AF1437FC7C552719B24AC47A5A080ACD'],
#                   2013: ['0B2E58DF2D0035EB19B32E50DA3C6D35',
#                          '8D6353FABEEED2FC0E283F3EB45B4270',
#                          '012119F37B6516E2BF351B7A4C92B2D3',
#                          'D7C98181058D50CE216A0C611844F577',
#                          'A26916E6CDF6B70EBEAF6350A53DAC93',
#                          'CEE549B3DFB8077163A9AF2AF36BDE70'],
#                   2012: ['A71837F881C01411BF8708550F89DF19',
#                          '0666FF7015B0F39DE8348F4C3D20CBF3',
#                          'DB0FF082E5B098944DB9FA64DE112873',
#                          'C03D4671EFD6CB984F49755FFCA59EAC',
#                          '20E3BA178F7907C5D2ED73524E2C7317',
#                          '642BF8AA4F4697C680A85A36C5FAD542'],
#                   2011: ['250E79960AFFD2E4AD5DD21D69EB6B46',
#                          'EBC96A2270FCDA02C95A393D7BD1442B',
#                          '9BF2921011AAD5C3AA97432B414C82AD',
#                          'DDA545C0676BB8F834B86859DC2EBAE5',
#                          'D6D91EBB7727283454C778BE7F4037B6',
#                          'D6BAA18AE5DE67998C4B56AE49B1DC51'],
#                   2010: ['AA29DAF3BAB334994A1B5E2856D7959E',
#                          'AFADB3F29D4C7CE7C254610B2FF0DA10',
#                          '53660414730E19B0D6391163EA0CD70C',
#                          'BD7D430FF4DFF39D26F55CB67C1E6338',
#                          'D4A8F967B7A32004FBEA40F2855B79F0']}

all_issues = get_bsObj("https://www.cambridge.org/core/journals/behavioral-and-brain-sciences/all-issues")
year_to_issues = defaultdict(list)
for bsObj in all_issues.find_all('span', text=re.compile("[0-9]{4} \- Volume ")):
    year = int(bsObj.text[:4])
    block = bsObj.parent.parent.parent.find('ul', class_='accordion level fourth')
    for single_issue in block.find_all('a', class_='row'):
        year_to_issues[year].append(single_issue['href'].split('/')[-1])

In [10]:
if not os.path.exists('articles/articles.json'):
    year_to_articles = {}
else:
    with open('articles/articles.json', 'r') as f:
        year_to_articles = json.load(f)

for year, issues in year_to_issues.items():
    if str(year) in year_to_articles:
        continue
    else:
        article_infos_whole_year = []
        for issue in issues:
            print('Downloading issue id: %s of year %d' % (issue, year))
            article_infos = get_article_infos_from_issue(issue)
            article_infos_whole_year.extend(article_infos)
        year_to_articles[str(year)] = article_infos_whole_year

with open('articles/articles_united.json', 'w') as f:
    json.dump(year_to_articles, f, ensure_ascii=False, indent=4)


 Erratum

Self-sacrifice for ingroup's history: A diachronic perspective—ERRATUM

 Target Article

Brain disorders? Not really: Why network structures block reductionism in psychopathology research

 Open Peer Commentary

Reductionist thinking and animal models in neuropsychiatric research
Beyond trait reductionism: Implications of network structures for dimensional models of psychopathology
The adaptive self: Culture and social flexibility in feedback networks
Beyond reduction with the representation: The need for causality with full complexity to unravel mental health
Symptoms are not the solution but the problem: Why psychiatric research should focus on processes rather than symptoms
Networks, intentionality and multiple realizability: Not enough to block reductionism
Indeed, not really a brain disorder: Implications for reductionist accounts of addiction
Conceptualizing neurodevelopmental disorders as networks: Promises and challenges
The value of clinical and translational neuros

Expanding the critique of the social motivation theory of autism with participatory and developmental research
Social motivation in autism: Gaps and directions for measurement of a putative core construct
Pragmatics and social motivation in autism
“Autistic people”? Who do you mean?
Two sources of bias affecting the evaluation of autistic communication
Compensation in autism is not consistent with social motivation theory
The double empathy problem, camouflage, and the value of expertise from experience

 Open Peer Commentary

The loneliness of me: The assumption of social disinterest and its worrying consequences in autism
Normocentric biases taint cognitive neuroscience and intervention of autism
Individual differences, social attention, and the history of the social motivation hypotheses of autism
A call for revamping socio-emotional ability research in autism
Social motivation in children with autism: Support from attachment research
To be or not to be emotionally aware and sociall

“Truth be told” – Semantic memory as the scaffold for veridical communication
Episodic memory and the witness trump card
More to episodic memory than epistemic assertion: The role of social bonds and interpersonal connection
Encoding third-person epistemic states contributes to episodic reconstruction of memories
Carving event and episodic memory at their joints
Episodic memory solves both social and nonsocial problems, and evolved to fulfill many different functions

 Open Peer Commentary

Using episodic memory to gauge implicit and/or indeterminate social commitments
Autonoesis and reconstruction in episodic memory: Is remembering systematically misleading?
Autonoesis and dissociative identity disorder
Epistemic authority, episodic memory, and the sense of self
False memories, nonbelieved memories, and the unresolved primacy of communication
Developmental roots of episodic memory
Enhanced action control as a prior function of episodic memory
Misconceptions about adaptive function
Con


 Open Peer Commentary

Replication is already mainstream: Lessons from small-N designs
Enhancing research credibility when replication is not feasible
Verify original results through reanalysis before replicating
Direct replications in the era of open sampling
You are not your data
The importance of exact conceptual replications
The replicability revolution
Constraints on generality statements are needed to define direct replication
What the replication reformation wrought
Verifiability is a core principle of science
What have we learned? What can we learn?
Conceptualizing and evaluating replication across domains of behavioral research
Making prepublication independent replication mainstream
Scientific progress is like doing a puzzle, not building a wall
Holding replication studies to mainstream standards of evidence
Data replication matters to an underpowered study, but replicated hypothesis corroboration counts

 Authors' Response

Improving social and behavioral science by making 

Are stereotypes accurate? A perspective from the cognitive science of concepts
Trustworthiness perception at zero acquaintance: Consensus, accuracy, and prejudice
Perceptions versus interpretations, and domains for self-fulfilling prophesies
The expressive rationality of inaccurate perceptions
Realism and constructivism in social perception
An evolutionary approach to accuracy in social perception
Intelligence, competitive altruism, and “clever silliness” may underlie bias in academe
Why would we expect the mind to work that way? The fitness costs to inaccurate beliefs
Stereotypes violate the postmodern construction of personal autonomy
Accurate perceptions do not need complete information to reflect reality
Choosing the right level of analysis: Stereotypes shape social reality via collective action
The social neuroscience of biases in in-and-out-group face processing
A close consideration of effect sizes reviewed by Jussim (2012)
There is more to memory than inaccuracy and distortion


...

“Prepared fears” and the theory of conditioning
Short-latency avoidance responses
What and where is the unconditioned (or conditioned) stimulus in the conditioning model of neurosis?

 Author's Response

The conditioning theory of neurosis: criticisms considered

 Target Article

Brain mechanisms for offense, defense, and submission

 Open Peer Commentary

The consociate modulator
Avian data on aggression
Limits of neurophysiological approaches to aggression
Cerebellar contributions to response selection
Neurobehavioral systems for attack and defense
Dividing up aggression and considerations in studying the physiological substrates of these phenomena
Neurotransmitter organization of aggressive behavior
Cerebral play of forces in offensive-defensive mechanisms
Brain mechanisms of aggression: Dilemmas of perspective
Motives: Metaphors in motion
Is there anything new in the neurophysiology of aggression for social psychologists?
Androgens and aggression

 Open Peer Commentary

The advanta

Oculomotor hysteresis: implications for testing sensorimotor and ecological optics theories
Motor system changes are not necessary for changes in perception
Methodological considerations in replicating Held and Rekosh's perceptual adaptation study
The thesis of the efference-mediation of vision cannot be rationalized
Visuomotor feedback: A short supplement to Gyr's journey around a polka-dotted cylinder
Attentional factors in depth perception

 Open Peer Commentary

Three functions of motor-sensory feedback in object perception
Separating the issues involved in the role of bodily movement in perception and perceptual-motor coordination

 Author's Response

Motor factors in perception

 Target Article

Homeostasis and drinking

 Open Peer Commentary

Multiple paths in the control of drinking
Roles of taste and learning in water regulation
Toy rats and real rats: nonhomeostatic plasticity in drinking
Is thirst largely an acquired specific appetite?
Broadening the homeostatic concept
Intr

Mind-brain; Puccetti & Dykes' non-solution to a non-problem
The insignificance of incommensurate variations
Are the properties of cells relevant for understanding consciousness?
Cortical localization and the mind-brain identity theory
Mentalist monism: consciousness as a causal emergent of brain processes
A false alternative
Codes, sensations, and the mind-body problem
The mind-brain issue unsimplified
Leibniz, location, and distinguishing types of sensation

 Author's Response

Localizationism and dualism: a second look at the paradox

 Target Article

Interpersonal expectancy effects: the first 345 studies

 Open Peer Commentary

The combined probabilities of 345 studies: only half the story?
On the biases of psychologists
Expecting expectancy effects: biased data analyses and failure to exclude alternative interpretations in experimenter expectancy research
On the misuse of statistics
Inattention to expectancy: resistance to a knowledge claim
Replication of experiments: a sociologic

Agnostic gauges and Genevan stages
“Measurement sequences,” Piagetian structures, and high-order rules
Is Piaget a Pied Piper?
Measurement sequences, logical necessity, and common sense
Holistic aspects of the stage notion
In defence of circularity
Which comes first – describing or explaining?
Conjoint construct validation

 Author's Response

Invariant sequences, explanation, and other stage criteria: reflections and replies

 Editorial

Editorial (Part II)

 Target Article

The nature and plausibility of Cognitivism

 Open Peer Commentary

On inference from input/output
On making distinctions that are not maintained
The scope of cognitivism
On explanation, holograms, moods, and skills
How to register dissatisfaction with A.I.
Systems and cognitive capacities
Co-opting holograms
Cognitive problems and problems of cognitivism
Cognitivism vs. Hermeneutics

 Open Peer Commentary

Mind that last step; I think it's loose
Systems and explanations
Half-way to realism: Some sympathetic commen

Signal learning and response learning
Decision rules, decision rules
Pexgo: a plausible construct in need of data
On explanation, familiarity, and response sequencing
Intelligent vs. stupid behavior, or learning concepts vs. responses
Limits on reinterpreting instrumental conditioning in terms of classical conditioning
Reinforcement is dead; long live reinforcement theories
Reinforcement theory: an experimenter-oriented rather than a subject-oriented philosophy
Learning theory, a will-o-the-wisp?
Implications for clinical behavior modification
A framework with no foundation: comments on Bindra's perceptual-motivation theory of response production

 Open Peer Commentary

Feedback systems invite tautological constructs
Stimulus theory and response theory: their complementarity and neurobehavioral basis
Behavioral evolution and individual development
Bindra's theory from the perspective of human motivation: unit size, stimulus centering, and the value of neural theory
A death too soon rep

In [11]:
for year, articles in year_to_articles.items():
    trees = transform_list_to_trees(articles)
    with open('articles/{}_articles.json'.format(year), 'w') as f:
        json.dump(trees, f, ensure_ascii=False, indent=4)

TypeError: tuple indices must be integers or slices, not str

In [12]:
with open('articles/articles_united.json', 'r') as f:
    all_year_to_articles = json.load(f)

In [18]:
for year, articles in all_year_to_articles.items():
    print(year)
    trees = transform_list_to_trees(articles)
    with open('articles/{}_articles.json'.format(year), 'w') as f:
        json.dump(trees, f, ensure_ascii=False, indent=4)

2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
{}


AssertionError: 